<a href="https://colab.research.google.com/github/treyra/CS155_miniproject/blob/data_visualization_v1/Tuning_Sergei_Decision_Trees_BAGGING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook attacks the learning problem with decision trees bagging

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
# READ THE TRAIN DATA:
df_train = pd.read_csv('https://raw.githubusercontent.com/treyra/CS155_miniproject/data_visualization_v1/data_cleanup/traindata_imputed_normalized_standard.csv',sep=',', header=0,index_col=0)
df_train.head()

,id,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,FIPS_CODE,SOURCE_REPORTING_UNIT_NAME,YEAR,DAY,LABEL,0,1,2,3
0,0,0.988292,-0.985738,1,0.054167,-0.026526,1.048773,-1.143120,-0.871063,0.0,0.002732,1,1.0,0.0,0.0,0.0
1,1,-0.401537,0.925104,-1,0.464583,-0.025770,-1.922012,0.386764,-0.852154,0.0,0.002732,4,0.0,0.0,0.0,1.0
2,2,-0.911691,1.066427,-1,0.666667,-0.026547,-1.906459,1.140289,-0.852154,0.0,0.002732,2,0.0,1.0,0.0,0.0
3,3,-0.814128,1.109280,-1,0.506250,-0.023489,-1.890905,-0.891945,-0.852154,0.0,0.002732,4,0.0,0.0,0.0,1.0
4,4,-0.448755,-0.759942,1,0.549583,-0.026456,0.675480,-0.503766,-0.833245,0.0,0.002732,2,0.0,1.0,0.0,0.0


In [3]:
# READ THE TEST DATA:
df_test = pd.read_csv('https://raw.githubusercontent.com/treyra/CS155_miniproject/data_visualization_v1/data_cleanup/testdata_imputed_normalized_standard.csv',sep=',', header=0,index_col=0)
df_test.head()

,id,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,FIPS_CODE,SOURCE_REPORTING_UNIT_NAME,YEAR,DAY
0,285382,-0.232600,-0.828839,1,0.66875,-0.026456,0.768803,-0.435263,-0.303789,1.058824,0.002732
1,285383,-0.335939,-0.760377,1,0.64375,-0.026526,0.784357,-0.435263,0.320212,1.058824,0.002732
2,285384,0.945131,-0.982574,1,0.50000,-0.026526,0.628819,-0.001415,-0.871063,1.058824,0.002732
3,285385,-0.817080,0.998962,-1,0.87500,-0.026314,-1.579828,0.500935,2.419125,1.058824,0.005464
4,285386,-0.682598,-0.695973,1,0.05250,-0.026526,1.204311,-0.960447,-0.681972,1.058824,0.005464


In [4]:
# DROP IRRELEVANT FEATURES:
df_test = df_test.drop(['id', 'FIPS_CODE', 'YEAR', 'SOURCE_REPORTING_UNIT_NAME','STATE'],axis=1)
df_train = df_train.drop(['id', 'FIPS_CODE', 'YEAR', 'SOURCE_REPORTING_UNIT_NAME','STATE'],axis=1)

data_arr = df_train.to_numpy()
X_train = data_arr[:,:-5]
y_train = data_arr[:,-5]
y_train = y_train.astype(int)
print("OUR ARRAY SHAPES:", X_train.shape, y_train.shape)
print(X_train[0])
df_train.head()


OUR ARRAY SHAPES: (285382, 6) (285382,)
[ 0.98829209 -0.98573763  0.05416667 -0.02652628  1.04877251  0.00273224]


,LATITUDE,LONGITUDE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,DAY,LABEL,0,1,2,3
0,0.988292,-0.985738,0.054167,-0.026526,1.048773,0.002732,1,1.0,0.0,0.0,0.0
1,-0.401537,0.925104,0.464583,-0.025770,-1.922012,0.002732,4,0.0,0.0,0.0,1.0
2,-0.911691,1.066427,0.666667,-0.026547,-1.906459,0.002732,2,0.0,1.0,0.0,0.0
3,-0.814128,1.109280,0.506250,-0.023489,-1.890905,0.002732,4,0.0,0.0,0.0,1.0
4,-0.448755,-0.759942,0.549583,-0.026456,0.675480,0.002732,2,0.0,1.0,0.0,0.0


# Gradient Boosting Model

In [42]:
from sklearn.model_selection import GridSearchCV
# create the classifier
params = {'n_estimators': [1000], 'max_samples':[0.08]}
#dt = DecisionTreeClassifier()
bc = BaggingClassifier(base_estimator=None, random_state=0) 

# Grid Search to determine best parameters
bc_grid = GridSearchCV(estimator=bc, param_grid=params, scoring='roc_auc_ovr', cv=5, n_jobs=1,return_train_score=True,verbose=1)
bc_grid.fit(X_train, y_train)
best_params = bc_grid.best_params_
print("Best: %f using %s" % (bc_grid.best_score_, bc_grid.best_params_))
means = bc_grid.cv_results_['mean_test_score']
stds = bc_grid.cv_results_['std_test_score']
params = bc_grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 22.5min finished


Best: 0.793512 using {'max_samples': 0.08, 'n_estimators': 1000}
0.793512 (0.005519) with: {'max_samples': 0.08, 'n_estimators': 1000}


In [ ]:
# results
# Best: 0.788644 using {'max_samples': 0.1, 'n_estimators': 120}

Best: 0.789117 using {'max_samples': 0.1, 'n_estimators': 130}
0.787652 (0.005505) with: {'max_samples': 0.05, 'n_estimators': 110}
0.788644 (0.005753) with: {'max_samples': 0.05, 'n_estimators': 130}
0.788231 (0.004916) with: {'max_samples': 0.1, 'n_estimators': 110}
0.789117 (0.004996) with: {'max_samples': 0.1, 'n_estimators': 130}
#
Best: 0.789381 using {'max_samples': 0.11, 'n_estimators': 135}
0.788872 (0.004915) with: {'max_samples': 0.1, 'n_estimators': 125}
0.789333 (0.004975) with: {'max_samples': 0.1, 'n_estimators': 135}
0.789019 (0.005209) with: {'max_samples': 0.11, 'n_estimators': 125}
0.789381 (0.005281) with: {'max_samples': 0.11, 'n_estimators': 135}

Best: 0.741987 using {'base_estimator__max_depth': 6, 'base_estimator__max_leaf_nodes': 15, 'n_estimators': 100}
0.720547 (0.007987) with: {'base_estimator__max_depth': 6, 'base_estimator__max_leaf_nodes': 10, 'n_estimators': 80}
0.720345 (0.007639) with: {'base_estimator__max_depth': 6, 'base_estimator__max_leaf_nodes': 10, 'n_estimators': 100}
0.741456 (0.004939) with: {'base_estimator__max_depth': 6, 'base_estimator__max_leaf_nodes': 15, 'n_estimators': 80}
0.741987 (0.003970) with: {'base_estimator__max_depth': 6, 'base_estimator__max_leaf_nodes': 15, 'n_estimators': 100}

Best: 0.789802 using {'max_samples': 0.08, 'n_estimators': 140}
0.785932 (0.005097) with: {'max_samples': 0.06, 'n_estimators': 80}
0.787722 (0.005362) with: {'max_samples': 0.06, 'n_estimators': 100}
0.789776 (0.005614) with: {'max_samples': 0.06, 'n_estimators': 140}
0.786202 (0.005281) with: {'max_samples': 0.08, 'n_estimators': 80}
0.787842 (0.005545) with: {'max_samples': 0.08, 'n_estimators': 100}
0.789802 (0.005611) with: {'max_samples': 0.08, 'n_estimators': 140}

Best: 0.790036 using {'max_samples': 0.08, 'n_estimators': 150}. 
0.789883 (0.005616) with: {'max_samples': 0.08, 'n_estimators': 145}
0.790036 (0.005628) with: {'max_samples': 0.08, 'n_estimators': 150}
0.789524 (0.005051) with: {'max_samples': 0.1, 'n_estimators': 145}
0.789676 (0.005081) with: {'max_samples': 0.1, 'n_estimators': 150}

Best: 0.789877 using {'max_samples': 0.09, 'n_estimators': 150}
0.789820 (0.005725) with: {'max_samples': 0.07, 'n_estimators': 150}
0.789877 (0.005526) with: {'max_samples': 0.09, 'n_estimators': 150}
#
Best: 0.790174 using {'max_samples': 0.08, 'n_estimators': 155}  
0.789981 (0.005634) with: {'max_samples': 0.08, 'n_estimators': 148}
0.790174 (0.005665) with: {'max_samples': 0.08, 'n_estimators': 155}
#
Best: 0.790751 using {'max_samples': 0.08, 'n_estimators': 180} 
0.790380 (0.005573) with: {'max_samples': 0.08, 'n_estimators': 165}
0.790751 (0.005609) with: {'max_samples': 0.08, 'n_estimators': 180}
#
Best: 0.791724 using {'max_samples': 0.08, 'n_estimators': 250} ### current best
0.791724 (0.005582) with: {'max_samples': 0.08, 'n_estimators': 250}
#
Best: 0.792112 using {'max_samples': 0.08, 'n_estimators': 300}
0.792112 (0.005546) with: {'max_samples': 0.08, 'n_estimators': 300}
#
Best: 0.792937 using {'max_samples': 0.08, 'n_estimators': 500}
0.792937 (0.005648) with: {'max_samples': 0.08, 'n_estimators': 500}
#
Best: 0.793512 using {'max_samples': 0.08, 'n_estimators': 1000}
0.793512 (0.005519) with: {'max_samples': 0.08, 'n_estimators': 1000}

In [43]:
# save
# READ THE TEST DATA:

X_test = df_test.to_numpy()
print(X_test.shape)

y_test = bc_grid.predict_proba(X_test)
print('Predicted: %s' % y_test[0])
print(y_test)

# save as csv_file in sample submission format
# put it together in a dataframe and save
from google.colab import  drive

def save_csv(df, savestr):
  drive.mount("/drive", force_remount=True)
  df.to_csv('/drive/My Drive/CS155_miniproject_1/'+str(savestr)+'.csv')

df_test_sub = pd.read_csv('https://raw.githubusercontent.com/treyra/CS155_miniproject/data_visualization_v1/data_cleanup/testdata_imputed_normalized_standard.csv',sep=',', header=0,index_col=0)
df_test_sub.head()
df = pd.DataFrame(y_test)
predictions = pd.DataFrame(df_test_sub['id'])
predictions['P1']=y_test[:,0]
predictions['P2']=y_test[:,1]
predictions['P3']=y_test[:,2]
predictions['P4']=y_test[:,3]
predictions = predictions.set_index('id')
predictions.head()

(73035, 6)
Predicted: [0.031 0.416 0.125 0.428]
[[0.031 0.416 0.125 0.428]
 [0.077 0.399 0.146 0.378]
 [0.054 0.421 0.208 0.317]
 ...
 [0.051 0.432 0.031 0.486]
 [0.015 0.504 0.071 0.41 ]
 [0.014 0.369 0.159 0.458]]


,P1,P2,P3,P4
id,,,,
285382,0.031,0.4160,0.125,0.4280
285383,0.077,0.3990,0.146,0.3780
285384,0.054,0.4210,0.208,0.3170
285385,0.006,0.7030,0.192,0.0990
285386,0.049,0.3445,0.209,0.3975


In [44]:
save_csv(predictions,'predictions_best_bagging_cv0.793512')

Mounted at /drive
